In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pylab as plt

In [2]:
import time
import random

In [3]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
import seaborn as sns
sns.set()

In [6]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,learning_curve, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

In [7]:
from sklearn.model_selection import KFold, StratifiedKFold

In [8]:
train = pd.read_csv("C:\\Users\\1-10\\Desktop\\python\\데이콘\\train.csv", index_col=0)
test = pd.read_csv("C:\\Users\\1-10\\Desktop\\python\\데이콘\\test.csv", index_col=0)
sample_submission = pd.read_csv("C:\\Users\\1-10\\Desktop\\python\\데이콘\\sample_submission.csv", index_col=0)

In [9]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

#### outlier 제거

In [10]:
for column in train.columns[2:-1]:
    test_min_value = test[column].min()
    test_max_value = test[column].max()

    for column in train.columns[2:-1]:
        train = train.drop(index=train[(train[column] <= test_min_value -100) | (train[column] >= test_max_value +100)].index)

    train = train.reset_index(drop=True)
    
train.shape

(199880, 23)

#### 모델 돌리기 전 데이터 준비

In [11]:
train_X = train.drop(columns=['type', 'type_num'], axis=1)
train_y = train['type_num']
test_x = test

In [12]:
train_X.shape

(199880, 21)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(train_X, 
                                                    train_y, 
                                                    stratify = train_y,
                                                    test_size=0.2, 
                                                    random_state=1234)

#### stratifiedKFold 사용

In [14]:
# kfold = KFold(n_splits=6, random_state= 0, shuffle = True)
kfold_s = StratifiedKFold(n_splits=6, random_state= 0, shuffle = True)

#### model 돌리기

In [15]:
from lightgbm import LGBMClassifier

In [16]:
from bayes_opt import BayesianOptimization

In [17]:
import lightgbm as lgb

#### Bayesian Optimization

In [18]:
def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
#     params = {'application':'multiclass','num_iterations':4000, 'learning_rate':0.05, 'early_stopping_round':100, 'metric':'multi_logloss'}
    params = {'application':'multiclass','num_iterations':1000, 'learning_rate':0.15, 'early_stopping_round':500, 'metric':'multi_logloss','num_class':19}
    params["num_leaves"] = round(num_leaves)
    params['feature_fraction'] = max(min(feature_fraction, 1), 0)
    params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
    params['max_depth'] = round(max_depth)
    params['lambda_l1'] = max(lambda_l1, 0)
    params['lambda_l2'] = max(lambda_l2, 0)
    params['min_split_gain'] = min_split_gain
    params['min_child_weight'] = min_child_weight
    cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['multi_logloss'])
    return max(cv_result['multi_logloss'])

In [19]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (55,80),
                                        'feature_fraction': (0.1,0.3),
                                        'bagging_fraction': (0.3,0.5),
                                        'max_depth': (4,9),
                                        'lambda_l1': (0.1,0.3),
                                        'lambda_l2': (0.2,0.5),
                                        'min_split_gain': (0.1,0.3),
                                        'min_child_weight':(0.1,0.4) }, random_state=0)

In [20]:
ytrain_temp = y_train + 1

In [21]:
len(np.unique(y_train))

19

In [24]:
def bayes_parameter_opt_lgb(X, y, init_round=5, opt_round=30, n_folds=6, random_seed=1234, n_estimators=1000, learning_rate=0.15, output_process=False):
    # prepare data
    #train_data = lgb.Dataset(data=X, label=y, categorical_feature = categorical_feats, free_raw_data=False)
    # parameters
    train_data = lgb.Dataset(data=X, label=y,  free_raw_data=False)

    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
#         params = {'application':'multiclass','num_iterations': n_estimators, 'learning_rate':learning_rate, 'early_stopping_round':100, 'metric':'multi_logloss'}
        params = {'application':'multiclass','num_iterations': n_estimators, 'learning_rate':learning_rate, 'early_stopping_round':100, 'metric':'multi_logloss','num_class':19}

        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['multi_logloss'])
        #print(cv_result.keys())
        return max(cv_result['multi_logloss-mean'])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (55,80),
                                        'feature_fraction': (0.1,0.3),
                                        'bagging_fraction': (0.3,0.5),
                                        'max_depth': (4,9),
                                        'lambda_l1': (0.1,0.3),
                                        'lambda_l2': (0.2,0.5),
                                        'min_split_gain': (0.1,0.3),
                                        'min_child_weight':(0.1,0.4) }, random_state=0)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result_200219_2.csv")
    
    # return best parameters
#     print(lgbBO.res.keys())
#     print(lgbBO.res['max'].keys())
    
    #return lgbBO.res['max']['max_params']
    # return lgbBO
    
    return lgbBO.max["params"]

In [56]:
# opt_params.res

In [23]:
np.unique(ytrain_temp)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19], dtype=int64)

In [25]:
# -----> input
opt_params = bayes_parameter_opt_lgb(X_train, y_train, 
                                     init_round=5, #5
                                     opt_round=30, # 10
                                     n_folds=6, # 3
                                     random_seed=1234, 
                                     n_estimators=1000,# 100 
                                     learning_rate=0.14) # 0.05

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------


C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:502: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:507: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's multi_logloss: 0.378911 + 0.00467628
[400]	cv_agg's multi_logloss: 0.37632 + 0.00462828
|  1        |  1.782    |  0.4098   |  0.243    |  0.2206   |  0.3635   |  6.118    |  0.2938   |  0.1875   |  77.29    |


C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:502: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:507: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's multi_logloss: 0.377866 + 0.00438529
|  2        |  1.798    |  0.4927   |  0.1767   |  0.2583   |  0.3587   |  6.84     |  0.3777   |  0.1142   |  57.18    |


...

[200]	cv_agg's multi_logloss: 0.435338 + 0.00450979
[400]	cv_agg's multi_logloss: 0.405539 + 0.00438597
[600]	cv_agg's multi_logloss: 0.398753 + 0.00433561
[800]	cv_agg's multi_logloss: 0.396569 + 0.00431744
[1000]	cv_agg's multi_logloss: 0.395483 + 0.00426525
|  33       |  2.001    |  0.5      |  0.1      |  0.1      |  0.5      |  4.0      |  0.1      |  0.3      |  55.0     |


C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:502: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:507: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's multi_logloss: 0.435588 + 0.00472347
[400]	cv_agg's multi_logloss: 0.406033 + 0.00435151
[600]	cv_agg's multi_logloss: 0.399264 + 0.00407625
[800]	cv_agg's multi_logloss: 0.397042 + 0.00413092
[1000]	cv_agg's multi_logloss: 0.396069 + 0.00409094
|  34       |  2.002    |  0.3      |  0.1      |  0.1      |  0.2      |  4.0      |  0.1      |  0.3      |  65.69    |


C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:502: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\1-10\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:507: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's multi_logloss: 0.435946 + 0.00461441
[400]	cv_agg's multi_logloss: 0.406794 + 0.00417814
[600]	cv_agg's multi_logloss: 0.400172 + 0.00419353
[800]	cv_agg's multi_logloss: 0.397794 + 0.00425304
[1000]	cv_agg's multi_logloss: 0.396711 + 0.00418612
|  35       |  2.002    |  0.3      |  0.1      |  0.3      |  0.2      |  4.0      |  0.1      |  0.3      |  80.0     |


In [26]:
print(opt_params)

{'bagging_fraction': 0.5, 'feature_fraction': 0.1, 'lambda_l1': 0.1, 'lambda_l2': 0.2, 'max_depth': 4.0, 'min_child_weight': 0.1, 'min_split_gain': 0.3, 'num_leaves': 80.0}


In [ ]:
# def bayes_parameter_opt_lgb(X_train, y_train, init_round=15, opt_round=25, n_folds=kfold_s,
#                            random_seed=0, n_estimators=1000, learning_rate=0.15):
   
#     # prepare data
#     train_data=lgb.Dataset(data=X_train, label=y_train, categorical_feature=list(X_train.columns),
#                           free_raw_data=False)
    
#     # parameters
#     def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2,
#             min_split_gain, min_child_weight):
#         params={'application':'multiclass', 'num_iterations':n_estimators, 
#                 'learning_rate':0.15, 'early_stopping_round':100, 'metric':'logloss'}
#         params['num_leaves']=round(num_leaves)
#         params['feature_fraction']=max(min(feature_fraction,1),0)
#         params['bagging_fraction']=max(min(bagging_fraction,1),0)
#         params['max_depth']=round(max_depth)
#         params['lambda_l1']=max(lambda_l1,0)
#         params['lambda_l2']=max(lambda_l2,0)
#         params['min_split_gain']=min_split_gain
#         params['min_child_weight']=min_child_weight
#         cv_result=lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval=200,
#                         metrics=['neg_log_loss'])
#         return max(cv_result('logloss'))
    
#     # range
#     lgbB0=BayesianOptimization(lgb_eval, {'num_leaves':64,
#                                          'feature_fraction':0.1,
#                                          'bagging_fraction':0.8,
#                                          'max_depth': 8,
#                                          'lambda_l1':0.5,
#                                          'lambda_l2':0.3,
#                                          'min_split_gain':0.001,
#                                          'min_child_weight':100} )
#     # optimize
#     lgbB0.maximize(init_points=init_round, n_iter=opt_round)
    
#     # output optimization process
#     if output_process==True : lgbB0.points_to_csv("bayes_opt_result.csv")
        
#     # return best parameters
#     return lgbB0.res['max']['max_params']

# opt_params=bayes_parameter_opt_lgb(train_X,train_y,init_round=5,opt_round=10,n_folds=kfold_s,
#                                   random_seed=0, n_estimators=1000, learning_rate=0.15)